In [1]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from sklearn.feature_selection import RFE

from sklearn.naive_bayes import BernoulliNB 
RANDOM_SEED = 42

# 读取数据

In [2]:
def unique_value(df: pd.DataFrame) -> np.array:
    df_specific = df.drop(list(df.columns[:2]), axis=1, inplace=False)
    temp = df_specific.to_numpy()
    return np.unique(temp)

In [3]:
train_df = pd.read_csv("../dataset/recipes_train.csv")
test_df = pd.read_csv("../dataset/recipes_test.csv")

In [4]:
train_df.shape, test_df.shape

((1469, 385), (979, 384))

In [5]:
train_df.head()

,id,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,1,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,chinese,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,chinese,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,korean,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,thai,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test_df.head()

,id,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# work

In [7]:
food_index = {}
food_count = {} # key为类, value为对应类的出现次数
loc_index = {} # key为类, value为其所有数据的索引
for _, group in train_df.groupby(by='cuisine'):
    # name, group = specific data -> DataFrame
    name = group['cuisine'].head(1).item()
    group.drop(["id", 'cuisine'], axis=1, inplace=True)
    loc_index[name] = group.index
    food_index[name] = group.sum().to_dict()
    food_count[name] = len(group)
food_index.keys() # key为类, value为类中各元素总和 dict: (column:sum_value)

dict_keys(['chinese', 'indian', 'japanese', 'korean', 'thai'])

In [8]:
# 将用于训练的内容提取出来
train_x = train_df.drop(columns=["cuisine"]).values
train_y = train_df["cuisine"].values
# 分割训练集验证集, 42更好。。
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [10]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(BernoulliNB(), \
                     param_grid={
                         "binarize":np.linspace(0.1, 0.9, 9)}, cv=4)
grid.fit(train_x, train_y)

GridSearchCV(cv=4, estimator=BernoulliNB(),
             param_grid={'binarize': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])})

In [15]:
BNB = BernoulliNB(binarize = .1)
BNB.fit(X_train, y_train)

print("train accuracy:", accuracy_score(BNB.predict(X_train), y_train))
print("valid accuracy:", accuracy_score(BNB.predict(X_valid), y_valid))

train accuracy: 0.8127659574468085
valid accuracy: 0.7993197278911565


## PCA

In [18]:
from sklearn.decomposition import PCA
best_result = 0
for n_component in [50, 100, 150, 200, 250, 300, 350]:
    features_names = train_df.columns[2:]
    labels = train_df.loc[:, 'cuisine']
    pca = PCA(n_components=n_component)
    reduced_features = pca.fit_transform(train_df.loc[:, features_names])
    train_x = reduced_features
    train_y = train_df["cuisine"].values
    grid = GridSearchCV(BernoulliNB(), \
                     param_grid={
                         "binarize":np.linspace(0.1, 0.9, 9)}, cv=4)
    grid.fit(train_x, train_y)
    if grid.best_score_ > best_result:
        best_result = grid.best_score_
        best_estimator = grid.best_estimator_
        best_n_component = n_component

In [21]:
best_result

0.6780354075346524